# Import a inicializace

In [ ]:
import copy                                  # Kopirovani prvku
import numpy as np                           # Matematicke operace
import random                                # Vyber nahodnych prvku
import sys                                   # Pripojeni knihoven (pro servery)
sys.path.append('/home/xbucha02/libraries')  # Pripojeni knihoven (pro servery)
import gym                                   # Prostredi Open AI Gym
#from gym import wrappers                    # Nahravani zaznamu
env = gym.make('CartPole-v0')                # Konkretni hra z Open AI Gym
#env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True)

%matplotlib inline  
import matplotlib.pyplot as plt              # Grafy
from collections import deque                # Pamet
from __future__ import division              # Deleni realnych cisel (kvuli nizsi verzi Pythonu 2.6)
from tqdm import tnrange, tqdm_notebook      # Progress bar
from profiling import * #profiling - @do_profile(follow=[method, ])

#Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics

#Tensorflow session
#gpuMemoryUsage=1                            # Vyuziti pameti GPU
#import tensorflow as tf                     # Knihovna Tensorflow
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = gpuMemoryUsage
#set_session(tf.Session(config=config))

# Parametry
observetime = 200                            # Delka pozorovani
episodes = 2000                                 # Pocet epizod
games = 100                                  # Pocet her
scores = []                                  # Pole pro ulozeni vysledku na analyzu
episodesList = []                            # Pole pro ulozeni cisel epizod na analyzu
#bestScore = 0                                # Promenna pro ukladani nejlepsiho prubezneho vysledku
actionCount = env.action_space.n             # Pocet vstupu do prostredi
stateSize = env.observation_space.shape[0]   # Pocet vystupu z prostredi

# Agent

In [ ]:


class Agent:
    def __init__(self, env):
        self.startEpsilon = 1                           # Pravdepodobnost konani nahodneho tahu na zacatku
        self.endEpsilon = 0.005                          # Pravdepodobnost konani nahodneho tahu na konci
        self.currentEpsilon = self.startEpsilon         # Soucasna pravdepodobnost konani nahodneho tahu
        self.epsilonDiminution = (self.startEpsilon - self.endEpsilon) / 50000#(self.startEpsilon - self.endEpsilon)/(episodes * 2)                  # Hodnota snizovani epsilonu
        self.gamma = 0.99                               # Discount faktor
        self.minibatchSize = 64                         # Velikost minibatche
        self.actionCount = env.action_space.n           # Pocet vstupu do prostredi
        self.stateSize = env.observation_space.shape[0] # Pocet vystupu z prostredi
        self.learningRate = 0.001                       # Learning rate
        self.fractionUpdate = 0.125
        self.memorySize = 10000                          # Velikost Replay memory
        self.primaryMemory = deque(maxlen=self.memorySize)
        self.secondaryMemory = deque(maxlen=self.memorySize)
        
        self.net = self.getNN(env)
        self.netTarget = self.getNN(env)
        self.updateTargetNet()

    def getNN(self, env):
        net = Sequential()
        net.add(Dense(32, activation="relu", input_shape=(self.stateSize,)))
        #net.add(Flatten())
        net.add(Dense(16, activation="relu"))
        
        net.add(Dense(self.actionCount, activation="linear"))

        net.summary()

        net.compile(loss=losses.mean_squared_error, optimizer=optimizers.Adam(lr=self.learningRate), metrics=['accuracy'])

        return net
    
    def updateTargetNet(self):
        self.netTarget.set_weights(self.net.get_weights())
    
    def updateTargetNetPartially(self):
        weights = self.net.get_weights()
        weightsTarget = self.netTarget.get_weights()
        
        for i in range(len(weightsTarget)):
            weightsTarget[i] = weights[i] * self.fractionUpdate + weightsTarget[i] * (1 - self.fractionUpdate)
            
        self.netTarget.set_weights(weightsTarget)
        
    def rememberPrimMem(self, state, action, reward, nextState, done):
        self.primaryMemory.append((state, action, reward, nextState, done))
    
    def rememberSecMem(self, state, action, reward, nextState, done):
        self.secondaryMemory.append((state, action, reward, nextState, done))
        
    def clearSecMem(self):
        self.secondaryMemory.clear()
        
    def epsilonActulization(self):
        if self.currentEpsilon > self.endEpsilon:
            if (self.currentEpsilon - self.epsilonDiminution) > self.endEpsilon:
                self.currentEpsilon = self.currentEpsilon - self.epsilonDiminution
            else:
                self.currentEpsilon = self.endEpsilon
    
    def getActionWE(self, state):
        if np.random.rand() <= self.currentEpsilon:
            return np.random.randint(0, self.actionCount, size=1)[0]
        else:
            Q = self.net.predict(state)
            return np.argmax(Q)

    def getAction(self, state):
        Q = self.net.predict(state)
        return np.argmax(Q)
    
    def resetSecMem(self):
        self.secondaryMemory = deque(maxlen=self.memorySize)
        
    def resetEpsilon(self):
        self.currentEpsilon = self.startEpsilon
    
    def trainDQN(self):
        if len(self.primaryMemory) >= self.minibatchSize:
            minibatch = random.sample(self.primaryMemory, self.minibatchSize) #z D vybere pocet mb_size samplu
        else:
            return
        
        #states = np.empty(self.minibatchSize)
        #print("{}" .format(states))
        #target_fs = np.empty(self.minibatchSize)
        state = np.vstack([i[0] for i in minibatch])
        action = [i[1] for i in minibatch]
        reward = [i[2] for i in minibatch]
        nextState = np.vstack([i[3] for i in minibatch])
        done = [i[4] for i in minibatch]
        

        target_fs = []
        
        for i in range(0, self.minibatchSize):
            #state = minibatch[i][0]
            #action = minibatch[i][1]
            #reward = minibatch[i][2]
            #nextState = minibatch[i][3]
            #done = minibatch[i][4]

            target_f = self.net.predict(state[i])

            if done:
                target_f[0][action[i]] = reward[i]
            else:
                aNet = self.net.predict(nextState[i])[0]

                target_f[0][action[i]] = reward + self.gamma * np.max(aNet)
            target_fs.append(target_f)
            
        self.net.fit(state, target_fs, epochs=1, verbose=0)
                
    def trainDDQN(self):
        if len(self.primaryMemory) >= self.minibatchSize:
            minibatch = random.sample(self.primaryMemory, self.minibatchSize) #z D vybere pocet mb_size samplu
        else:
            return

        state = np.vstack([i[0] for i in minibatch])
        action = [i[1] for i in minibatch]
        reward = [i[2] for i in minibatch]
        nextState = np.vstack([i[3] for i in minibatch])
        done = [i[4] for i in minibatch]

        target_fs = np.zeros((self.minibatchSize, self.actionCount))

        for i in range(0, self.minibatchSize):

            target_f = self.net.predict(np.array([state[i]]))
            #print("target_f: {}" .format(target_f))
            #print("{}" .format(done[i] ))
            if done[i] == 1:
                target_f[0][action[i]] = reward[i]
                #print("1:{}".format(minibatch[i]))
            else:
                aNet = self.net.predict(np.array([nextState[i]]))[0]
                #print("{}" .format(aNet))
                tNet = self.netTarget.predict(np.array([nextState[i]]))[0]
                target_f[0][action] = reward[i] + self.gamma * tNet[np.argmax(aNet)]
                #print("2")
            
            target_fs[i][0] = target_f[0][0]
            target_fs[i][1] = target_f[0][1]
        #print("State: {}, Target: {}" .format(state, target_fs))
        self.net.fit(state, target_fs, batch_size = self.minibatchSize, verbose = 0, epochs = 32)
    
    def loadNN(self, name):
        self.net.load_weights(name)

    def saveNN(self, name):
        self.net.save_weights(name)
        
    def profile(self):
        %lprun -f updateTargetNet()

# Algoritmus

In [ ]:
def gameTest(env, agent):
    
    total_reward = 0
    
    for game in range(games):

        state = env.reset()
        
        done = False
        
        while not done:

            state = np.reshape(state, (1, stateSize))
            
            action = agent.getActionWE(state) 

            nextState, reward, done, info = env.step(action)

            state = nextState
            
            total_reward = total_reward + reward

    return (total_reward / games)

In [ ]:

agent = Agent(env) #vytvoreni agenta
    
#agent.loadNN("./DDQN-MountainCar-v0.h5") #nacteni NN
#agent.updateTargetNet() #nacteni NN do netTarget
#
def maine():
    for eps in tnrange(episodes, desc='episodes'):

        state = env.reset() #resetovani prostredi

        for t in range(observetime):

            state = np.reshape(state, (1, stateSize))
            action = agent.getActionWE(state) #ziskani akce

            nextState, reward, done, info = env.step(action) #provedeni akce

            agent.rememberPrimMem(state, action, reward, nextState, done) #ulozeni do primarni pameti

            agent.trainDDQN() #trenovani na primarni pameti

            state = nextState #zmena stavu

            
            if eps % 50 == 0:
                agent.saveNN("./DDQN-CartPole-v0.h5") #ulozeni site
            agent.epsilonActulization()


                
            if done: #konec epizody
                #print("Episode: {}/{}, epsilon: {:.2}, score: {}".format(eps, episodes, agent.currentEpsilon, t))
                agent.updateTargetNet() #aktualizace target site
                scores.append(t) #ulozeni aktualniho skore
                episodesList.append(eps) #ulozeni aktualniho cisla epizody

                if eps % 50 == 0:
                    print("Average score: {}".format(gameTest(env, agent)))
                #if t > bestScore: #bylo dosazeno njelepsiho skore
                #    bestScore = t #nove nejlepsi skore
                #    for i in range(trainingAfterSucces): #pocet trenovani
                #        agent.trainDDQN(2) #trenovani na sekundarni pameti
                #agent.saveNN("./DDQN-CartPole-v0-{}.h5" .format(eps)) #ulozeni site
                #    #agent.resetEpsilon() #resetovani epsilonu

                #agent.resetSecMem() #vycisteni sekundarni pameti

                break
        #sleep(0.01)

    agent.saveNN("./DDQN-CartPole-v0.h5") #ulozeni site

maine()

# Výsledky

In [ ]:

import copy

def analysis(scores, episodesList):
    score1 = copy.copy(scores)
    score2 = copy.copy(scores)
    score3 = copy.copy(scores)

    for i in range (len(scores)):
        if i > 1 and i < (len(scores)-2):
            score1[i] = (scores[i - 2] + scores[i - 1] + scores[i] + scores[i + 1] + scores[i + 2])/5

    for i in range (len(scores)):
        if i > 4 and i < (len(scores)-5):
            score2[i] += scores[i - 5] + scores[i - 4] + scores[i - 3] + scores[i - 2] + scores[i - 1]
            score2[i] += scores[i + 5] + scores[i + 4] + scores[i + 3] + scores[i + 2] + scores[i + 1]
            score2[i] = score2[i]/11

    for i in range (len(scores)):
        if i > 49 and i < (len(scores) - 50):
            for e in range (1,50):
                score3[i] += scores[i - e] + scores[i + e] 
            score3[i] = score3[i]/101      

    plt.plot(episodesList, scores, 'ro')
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Vysledky")
    plt.show()

    plt.plot(scores)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku")
    plt.show()

    plt.plot(score1)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku (filtr - prumer 5-ti prvku)")
    plt.show()

    plt.plot(score2)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku  (filtr - prumer 11-cti prvku)")
    plt.show()

    plt.plot(score3)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku  (filtr - prumer 101 prvku)")
    plt.show()
    
analysis(scores, episodesList)

# Hra